<div style="font-size: 22px; line-height: 1.6;">

# 🌊 Marine Carbonate Visualizer

<div style="text-align: center; margin: 30px 0;">
    <img src="figures/carbonate_system.svg" width="800" alt="Marine Carbonate System Diagram" style="border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); max-width: 100%;">
</div>

</div>



<div style="font-size: 22px; line-height: 1.6;">

---

## 📘 *Computing the Carbonate System in Seawater*

This function allows you to calculate key parameters of the marine **carbonate system** using the Python package **PyCO2SYS**, a standard tool in oceanography.

---

#### 🧮 What does the function compute?

Given:

* **Total alkalinity** (TA) in µmol/kg
* **Dissolved Inorganic Carbon** (DIC) in µmol/kg

and assuming:

* Constant **salinity** (default = 35 PSU)
* Constant **temperature** (default = 25 °C)

The function returns:

* **pH** (on the total scale)
* **pCO₂** (partial pressure of CO₂ in µatm)
* **\[HCO₃⁻]** (bicarbonate ion concentration)
* **\[CO₃²⁻]** (carbonate ion concentration)

</div>


In [16]:
import PyCO2SYS as pyco2

def compute_carbonate_system(alkalinity, DIC, salinity=35, temperature=25):
    # Run CO2SYS using alkalinity and DIC as input parameters
    results = pyco2.sys(
        par1=alkalinity,      # par1_type = 1 → Total Alkalinity (µmol/kg)
        par2=DIC,             # par2_type = 2 → DIC = Dissolved Inorganic Carbon (µmol/kg)
        par1_type=1,          # 1 = total alkalinity
        par2_type=2,          # 2 = DIC
        salinity=salinity,    # Salinity in PSU
        temperature=temperature  # Temperature in °C
    )

    # Return selected key results in a dictionary
    return {
        "pH_total": results["pH_total"],         # pH on the total scale
        "pCO2": results["pCO2"],                 # partial pressure of CO₂ (µatm)
        "bicarbonate": results["bicarbonate"],   # HCO₃⁻ concentration (µmol/kg)
        "carbonate": results["carbonate"],       # CO₃²⁻ concentration (µmol/kg)
        "omega_aragonite": results["saturation_aragonite"],  # saturation state with respect to aragonite
        "salinity": salinity,
        "temperature": temperature
    }



In [17]:
from IPython.display import display, HTML

def show_conditions(data):
    html = f"""
    <div style="font-size: 20px; color: #00204C; font-family: sans-serif; margin-top: 15px;">
        <p><strong>🧪 Conditions used in the calculation:</strong></p>
        <p>🔹 Salinity: {data['salinity']:.0f} PSU</p>
        <p>🌡️ Temperature: {data['temperature']:.0f} °C</p>
    </div>
    """
    display(HTML(html))


In [18]:
from IPython.display import HTML

def display_indicators(data):
    # Extended Cividis palette with 5 colors
    colors = ["#00204C", "#40587C", "#A1B56C", "#FDE725", "#f98e09"]  
    
    labels = [
        ("pCO₂", data["pCO2"], ".0f", "µatm"),
        ("HCO₃⁻", data["bicarbonate"], ".0f", "µmol/kg"),
        ("CO₃²⁻", data["carbonate"], ".0f", "µmol/kg"),
        ("pH", data["pH_total"], ".2f", "total scale"), 
        ("Ωₐ", data["omega_aragonite"], ".2f", "aragonite saturation")
    ]
    
    # Create HTML display instead of plotly for better Voilà compatibility
    html = """
    <div style="font-family: Arial, sans-serif; padding: 25px; background: #f8f9fa; border-radius: 15px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
        <div style="display: flex; justify-content: space-around; flex-wrap: wrap; gap: 15px;">
    """
    
    for i, (label, value, fmt, unit) in enumerate(labels):
        formatted_value = f"{value:{fmt}}"
        html += f"""
            <div style="text-align: center; padding: 20px; background: white; border-radius: 12px; 
                        min-width: 140px; flex: 1; box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                        border-left: 5px solid {colors[i]};">
                <h3 style="color: {colors[i]}; margin: 0 0 10px 0; font-size: 18px;">{label}</h3>
                <p style="font-size: 42px; font-weight: bold; color: {colors[i]}; margin: 10px 0;">{formatted_value}</p>
                <small style="color: #666; font-size: 14px;">{unit}</small>
            </div>
        """
    
    html += """
        </div>
    </div>
    """
    
    return HTML(html)


In [19]:
from ipywidgets import FloatSlider, Layout, HTML, VBox, interact, Button
from IPython.display import display

# Slider style settings - increased spacing for labels
slider_style = {'description_width': '350px'}
slider_layout = Layout(width='950px', margin='0 0 20px 0', justify_content='flex-start')

# Alkalinity slider
alk_slider = FloatSlider(
    value=2300, min=1800, max=3500, step=25,
    description="Alkalinity (µmol/kg)",
    style=slider_style, layout=slider_layout,
    continuous_update=False,
    readout=True, readout_format=".0f"
)

# DIC slider
dic_slider = FloatSlider(
    value=2020, min=1600, max=3500, step=25,
    description="DIC (µmol/kg)",
    style=slider_style, layout=slider_layout,
    continuous_update=False,
    readout=True, readout_format=".0f"
)

# Custom CSS for widgets
custom_css = HTML("""
<style>
.widget-label {
    font-size: 18pt !important;
    font-weight: bold;
    color: #00204C !important;
}
.widget-readout {
    font-size: 20pt !important;
    font-weight: bold;
    color: #00204C !important;
}
input[type="range"] {
    height: 20px !important;
    background: linear-gradient(to right, #00204C, #FDE725) !important;
    border-radius: 10px;
}
</style>
""")

# Create a reset button
reset_button = Button(
    description='Reset Values',
    button_style='info',
    layout=Layout(width='150px', margin='10px 0 0 0')
)

def reset_values(b):
    alk_slider.value = 2300
    dic_slider.value = 2020

reset_button.on_click(reset_values)


In [20]:
from ipywidgets import Output, VBox, interactive_output, Button, Layout
from IPython.display import display, clear_output

# Create output widget to display results
output = Output()

# Function to update display when sliders change
def update_display(alkalinity, DIC):
    # Compute carbonate system variables using current slider values
    data = compute_carbonate_system(alkalinity, DIC)
    
    # Clear and update output
    with output:
        clear_output(wait=True)
        
        # Create and display the indicators
        indicators_html = display_indicators(data)
        display(indicators_html)
        
        # Show the constant salinity and temperature values used in the model
        show_conditions(data)

# Function to reset sliders to default values
def reset_values(button):
    alk_slider.value = 2300
    dic_slider.value = 2100  # Use consistent default value
    with output:
        clear_output(wait=True)
    update_display(alk_slider.value, dic_slider.value)

# Create reset button
reset_button = Button(
    description="🔄 Reset Values",
    button_style='info',
    layout=Layout(width='200px', margin='10px 0')
)
reset_button.on_click(reset_values)

# Create interactive output that connects sliders to update function
interactive_plot = interactive_output(update_display, {'alkalinity': alk_slider, 'DIC': dic_slider})

# Display custom CSS styles for consistent font and slider appearance
display(custom_css)

# Create the interface layout
interface = VBox([
    alk_slider,
    dic_slider,
    reset_button,
    output
])

# Display the interface
display(interface)

# Trigger initial update
update_display(alk_slider.value, dic_slider.value)



HTML(value='\n<style>\n.widget-label {\n    font-size: 18pt !important;\n    font-weight: bold;\n    color: #0…

**Author:** Cardoso-Mohedano JG 
**Institution:** Instituto de Ciencias del Mar y Limnología, UNAM, Estación El Carmen  
**License:** [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)  
**ORCID:** [0000-0002-2918-972X](https://orcid.org/0000-0002-2918-972X)

---
This interactive tool allows exploration of the marine carbonate system using [PyCO2SYS](https://pyco2sys.readthedocs.io/en/latest/).  
Developed with support from [Claude AI](https://claude.ai) by Anthropic and [OpenAI ChatGPT](https://openai.com/chatgpt) and educational Python tools.
